In [0]:
tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@kvjdb.dfs.core.windows.net/" for tier in tiers}

bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[]

In [0]:

import requests
import json
from datetime import date, timedelta

In [0]:
# Get base parameters
dbutils.widgets.text("start_date", "")
dbutils.widgets.text("end_date", "")
start_date = dbutils.widgets.get("start_date")
end_date = dbutils.widgets.get("end_date")

In [0]:
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"


In [0]:
url

'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2025-05-24&endtime=2025-05-25'

In [0]:

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")


Wrote 258443 bytes.
Data successfully saved to abfss://bronze@kvjdb.dfs.core.windows.net//2025-05-24_earthquake_data.json


In [0]:
data[0]

{'type': 'Feature',
 'properties': {'mag': 1.73,
  'place': '10 km  of Pittsburg, CA',
  'time': 1748130484280,
  'updated': 1748166738755,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75186056',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75186056&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 46,
  'net': 'nc',
  'code': '75186056',
  'ids': ',nc75186056,',
  'sources': ',nc,',
  'types': ',nearby-cities,origin,phase-data,scitech-link,',
  'nst': 16,
  'dmin': 0.03164,
  'rms': 0.13,
  'gap': 97,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 1.7 - 10 km  of Pittsburg, CA'},
 'geometry': {'type': 'Point',
  'coordinates': [-121.9165, 38.1118333333333, 25.87]},
 'id': 'nc75186056'}

In [0]:
# Define your variables
output_data = {
    "start_date": start_date,
    "end_date": end_date,
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Serialize the dictionary to a JSON string
output_json = json.dumps(output_data)

# Log the serialized JSON for debugging
print(f"Serialized JSON: {output_json}")

# Return the JSON string
dbutils.notebook.exit(output_json)